In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds


In [5]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

AttributeError: module 'tensorflow_datasets' has no attribute 'load'